# Rapport Nicolas CHARPENTIER


Le but de ce projet est d’analyser les différentes équipes composant ce championnat, leurs victoires, défaites, buts marqués et encaissés, et ensuite de pouvoir prédire certaines victoires et au finale, le possible gagnant du championnat.

Nous savons que les grands championnats européens sont très indécis et improbables, nous pensons en particulier au club de Leicester remportant le championnat d’Angleterre l’an dernier avec un budget très inférieur aux grands clubs anglais. Ce club connu pour le rugby qui venait directement de deuxième ligue anglaise. Il est aussi possible d’énoncer Montpellier en 2012, qui fût champion de France à la surprise générale devant le grandissime favoris (le paris saint germain).

C’est pour cela que nous allons nous intéresser à un championnat d’une division inférieur, car les statistiques sont beaucoup plus fiables du fait qu’il y a moins d’argent en jeux, moins de professionnalisme et moins de variables extérieures. 

J’ai choisi la deuxième division du championnat allemand : La Bundesliga 2.
J’ai choisi ce championnat car les championnats allemands ont pour réputation d’être stable et homogène, ce qui convient parfaitement pour nos analyses.

Ces analyses seront effectuées sur les années 2013-2014, 2014-2015, et 2015-2016.


In [803]:
import numpy as np
import scipy.stats as ss
import sklearn
import pandas as pd
%matplotlib inline
import csv as csv
import StringIO
import pydotplus
from sklearn.externals.six import StringIO 
from sklearn import datasets,tree
from matplotlib import pyplot


data2007 = pd.read_csv('Bundesliga2_2006_2007.csv', header=0)
data2008 = pd.read_csv('Bundesliga2_2007_2008.csv', header=0)
data2009 = pd.read_csv('Bundesliga2_2008_2009.csv', header=0)
data2010 = pd.read_csv('Bundesliga2_2009_2010.csv', header=0)
data2011 = pd.read_csv('Bundesliga2_2010_2011.csv', header=0)
data2012 = pd.read_csv('Bundesliga2_2011_2012.csv', header=0)
data2013 = pd.read_csv('Bundesliga2_2012_2013.csv', header=0)
data2014 = pd.read_csv('Bundesliga2_2013_2014.csv', header=0)
data2015 = pd.read_csv('Bundesliga2_2014_2015.csv', header=0)
data2016 = pd.read_csv('Bundesliga2_2015_2016.csv', header=0)
data2017 = pd.read_csv('Bundesliga2_2016_2017.csv', header=0)

# 1.Première visualisation des données et nettoyage.

Dans cette première partie nous allons effectuer une première visualisation des données

In [695]:
data2014.describe()
print(data2014.head(10))
data2014.dtypes

  Div      Date            HomeTeam        AwayTeam  FTHG  FTAG FTR  HTHG  \
0  D2  19/07/13          Ingolstadt  Erzgebirge Aue     1     2   A     0   
1  D2  19/07/13          Sandhausen           Aalen     0     0   D     0   
2  D2  19/07/13            St Pauli     Munich 1860     1     0   H     0   
3  D2  20/07/13             Dresden         FC Koln     1     1   D     0   
4  D2  20/07/13           Paderborn  Kaiserslautern     0     1   A     0   
5  D2  21/07/13       Frankfurt FSV       Karlsruhe     0     1   A     0   
6  D2  21/07/13      Greuther Furth       Bielefeld     2     0   H     2   
7  D2  21/07/13        Union Berlin          Bochum     1     2   A     0   
8  D2  22/07/13  Fortuna Dusseldorf         Cottbus     1     0   H     0   
9  D2  26/07/13               Aalen  Greuther Furth     0     2   A     0   

   HTAG HTR  ...   BbAv<2.5  BbAH  BbAHh  BbMxAHH  BbAvAHH  BbMxAHA  BbAvAHA  \
0     1   A  ...       1.75  17.0  -0.50     1.99     1.92     1.98     

Div          object
Date         object
HomeTeam     object
AwayTeam     object
FTHG          int64
FTAG          int64
FTR          object
HTHG          int64
HTAG          int64
HTR          object
B365H       float64
B365D       float64
B365A       float64
BWH         float64
BWD         float64
BWA         float64
IWH         float64
IWD         float64
IWA         float64
LBH         float64
LBD         float64
LBA         float64
PSH         float64
PSD         float64
PSA         float64
WHH         float64
WHD         float64
WHA         float64
SJH         float64
SJD         float64
SJA         float64
VCH         float64
VCD         float64
VCA         float64
Bb1X2         int64
BbMxH       float64
BbAvH       float64
BbMxD       float64
BbAvD       float64
BbMxA       float64
BbAvA       float64
BbOU          int64
BbMx>2.5    float64
BbAv>2.5    float64
BbMx<2.5    float64
BbAv<2.5    float64
BbAH        float64
BbAHh       float64
BbMxAHH     float64
BbAvAHH     float64


Nous voyons ici que beaucoup de colonnes, mais pour l'instant on va s'intéresser que à quelques unes pour nos premières analyses. + renommage

In [696]:
tt = data2014[['HomeTeam','AwayTeam']] 

tt = data2014[(data2014['HomeTeam']=='FC Koln') & (data2014['FTR']=='H') | (data2014['AwayTeam']=='FC Koln') & (data2014['FTR']=='A')]
#print(tt.count())

# 2.Analyse 

## 2.1. Analyse générale d'une saison


On sait que 3 descendent et 3 montent...

In [784]:
def nbVictoires(data, year):
    tab2014 = []
    for team in data['HomeTeam']:
        res = data[(data['HomeTeam']==team) & (data['FTR']=='H') | (data['AwayTeam']==team) & (data['FTR']=='A')]
        tab1 = [res['HomeTeam'].count() ,team]
        if tab1 not in tab2014:
            tab2014.append([res['HomeTeam'].count(), team ])
    tab2014.sort() 
    print('Nombres de victoire par equipe pour la saison ' + year + ' :')
    for i in tab2014:
        print(i)
        
def nbVictoiresDomicile(data):
    tab2014 = []
    for team in data['HomeTeam']:
        res = data[(data['HomeTeam']==team) & (data['FTR']=='H')]
        tab1 = [res['HomeTeam'].count() ,team]
        if tab1 not in tab2014:
            tab2014.append([res['HomeTeam'].count(), team ])
    tab2014.sort() 
    print('Nombres de victoire à domicile par equipe pour les 3 saisons :' )
    for i in tab2014:
        print(i)

def nbVictoiresExterieur(data):
    tab2014 = []
    for team in data['AwayTeam']:
        res = data[(data['AwayTeam']==team) & (data['FTR']=='A')]
        tab1 = [res['AwayTeam'].count() ,team]
        if tab1 not in tab2014:
            tab2014.append([res['AwayTeam'].count(), team ])
    tab2014.sort() 
    print('Nombres de victoire à extérieur par equipe pour les 3 saisons :' )
    for i in tab2014:
        print(i)

        
#Nombre de victoire 
df2012 = data2012[['HomeTeam','AwayTeam','FTR']]
df2013 = data2013[['HomeTeam','AwayTeam','FTR']]
df2014 = data2014[['HomeTeam','AwayTeam','FTR']]
df2015 = data2015[['HomeTeam','AwayTeam','FTR']]
df2016 = data2016[['HomeTeam','AwayTeam','FTR']]
alln = [data2007,data2008,data2009,data2010,data2011,data2012,data2013,data2014, data2015, data2016]
dataall = pd.concat(alln)
nbVictoires(df2012, '2012')
nbVictoires(df2013, '2013')
nbVictoires(df2014, '2014')
nbVictoires(df2015, '2015')
nbVictoires(df2016, '2016')


    

Nombres de victoire par equipe pour la saison 2012 :
[5, 'Hansa Rostock']
[6, 'Aachen']
[7, 'Frankfurt FSV']
[8, 'Cottbus']
[8, 'Erzgebirge Aue']
[8, 'Ingolstadt']
[9, 'Karlsruhe']
[10, 'Bochum']
[10, 'Braunschweig']
[10, 'Duisburg']
[12, 'Dresden']
[14, 'Union Berlin']
[16, 'Fortuna Dusseldorf']
[17, 'Munich 1860']
[17, 'Paderborn']
[18, 'St Pauli']
[20, 'Ein Frankfurt']
[20, 'Greuther Furth']
Nombres de victoire par equipe pour la saison 2013 :
[4, 'Regensburg']
[6, 'Sandhausen']
[9, 'Dresden']
[9, 'Erzgebirge Aue']
[10, 'Bochum']
[10, 'Ingolstadt']
[11, 'Duisburg']
[11, 'Paderborn']
[11, 'St Pauli']
[12, 'Aalen']
[12, 'Cottbus']
[12, 'Munich 1860']
[13, 'Union Berlin']
[14, 'FC Koln']
[15, 'Kaiserslautern']
[16, 'Frankfurt FSV']
[19, 'Braunschweig']
[22, 'Hertha']
Nombres de victoire par equipe pour la saison 2014 :
[5, 'Dresden']
[6, 'Cottbus']
[9, 'Bielefeld']
[11, 'Aalen']
[11, 'Bochum']
[11, 'Erzgebirge Aue']
[11, 'Frankfurt FSV']
[11, 'Ingolstadt']
[11, 'Union Berlin']
[12, 'Ka

On se rend compte que ... alors Nous allons nous focaliser sur les equipes ayant participes aux 3 saisons consecutives :

In [801]:


#recuperation des equipes de chaque saison

allteams2007 = data2007['HomeTeam']
teams2007 = []
for i in allteams2007:
    if i not in teams2007:
        teams2007.append(i)
print('Equipes de la saison 2007 : ')
print(teams2007)

allteams2008 = data2008['HomeTeam']
teams2008 = []
for i in allteams2008:
    if i not in teams2008:
        teams2008.append(i)
print('Equipes de la saison 2008 : ')
print(teams2008)

allteams2009 = data2009['HomeTeam']
teams2009 = []
for i in allteams2009:
    if i not in teams2009:
        teams2009.append(i)
print('Equipes de la saison 2009 : ')
print(teams2009)

allteams2010 = data2010['HomeTeam']
teams2010 = []
for i in allteams2010:
    if i not in teams2010:
        teams2010.append(i)
print('Equipes de la saison 2010 : ')
print(teams2010)

allteams2011 = data2011['HomeTeam']
teams2011 = []
for i in allteams2011:
    if i not in teams2011:
        teams2011.append(i)
print('Equipes de la saison 2011 : ')
print(teams2011)


allteams2012 = data2012['HomeTeam']
teams2012 = []
for i in allteams2012:
    if i not in teams2012:
        teams2012.append(i)
print('Equipes de la saison 2012 : ')
print(teams2012)

allteams2013 = data2013['HomeTeam']
teams2013 = []
for i in allteams2013:
    if i not in teams2013:
        teams2013.append(i)
print('Equipes de la saison 2013 : ')
print(teams2013)

allteams2014 = data2014['HomeTeam']
teams2014 = []
for i in allteams2014:
    if i not in teams2014:
        teams2014.append(i)
print('Equipes de la saison 2014 : ')
print(teams2014)

allteams2015 = data2015['HomeTeam']
teams2015 = []
for i in allteams2015:
    if i not in teams2015:
        teams2015.append(i)
print('Equipes de la saison 2015 : ')
print(teams2015)        

allteams2016 = data2016['HomeTeam']
teams2016 = []
for i in allteams2016:
    if i not in teams2016:
        teams2016.append(i)
print('Equipes de la saison 2016 : ')
print(teams2016)      


#Recuperation des equipes ayant participe a la bundesliga2 les 3 saisons
all_teams = sum([teams2010,teams2011,teams2012, teams2013, teams2014, teams2015, teams2016], [])
tab = []
kk = 0
for i in all_teams:
    if i not in tab:
        tab.append(i)
print('Equipes ayant participées aux 7 saisons :')
newtab = []
for i in tab:
    kk = all_teams.count(i)
    if kk >= 6:
        newtab.append(i)
print(newtab)

Equipes de la saison 2007 : 
['Burghausen', 'Kaiserslautern', 'Offenbach', 'Braunschweig', 'Duisburg', 'Erzgebirge Aue', 'Freiburg', 'Greuther Furth', 'Augsburg', 'CZ Jena', 'Hansa Rostock', 'Munich 1860', 'Essen', 'FC Koln', 'Koblenz', 'Paderborn', 'Unterhaching', 'Karlsruhe']
Equipes de la saison 2008 : 
['Aachen', 'Osnabruck', 'St Pauli', 'Augsburg', 'Erzgebirge Aue', 'Hoffenheim', 'Mainz', 'Paderborn', 'Kaiserslautern', 'Freiburg', 'Koblenz', 'Wehen', 'CZ Jena', 'Greuther Furth', "M'gladbach", 'Munich 1860', 'Offenbach', 'FC Koln']
Equipes de la saison 2009 : 
['Aachen', 'Mainz', 'St Pauli', 'Ahlen', 'Freiburg', 'Ingolstadt', 'Koblenz', 'Nurnberg', 'Duisburg', 'Frankfurt FSV', 'Greuther Furth', 'Osnabruck', 'Augsburg', 'Hansa Rostock', 'Munich 1860', 'Oberhausen', 'Wehen', 'Kaiserslautern', 'Kaiserslautern ']
Equipes de la saison 2010 : 
['Karlsruhe', 'Oberhausen', 'St Pauli', 'Fortuna Dusseldorf', 'Kaiserslautern', 'Cottbus', 'Frankfurt FSV', 'Munich 1860', 'Bielefeld', 'Duisburg'

On va donc s'interesser un peu plus à ces équipes la. On se rend comtpe que avec le k = 5 ou 6 ect on a les équipes qui ont joué sur les 5 ou 6 derniere années dans cette division. Exemple : 3 équipes sur les 7 dernieres années

## 2.2. Anayse particulière sur quelques équipes ressortis dans l'analyse précédente

Nous allons créer un dataset incluant toutes les données de départ uniquement sur ces 2 équipes : Pour cela reprenons notre newtab

In [738]:
for i in data2014['HomeTeam']:
    if i not in newtab:
        data2014 = data2014[data2014['HomeTeam'] != i]

for i in data2014['AwayTeam']:
    if i not in newtab:
        data2014 = data2014[data2014['AwayTeam'] != i]

for i in data2015['HomeTeam']:
    if i not in newtab:
        data2015 = data2015[data2015['HomeTeam'] != i]

for i in data2015['AwayTeam']:
    if i not in newtab:
        data2015 = data2015[data2015['AwayTeam'] != i]
        
for i in data2016['HomeTeam']:
    if i not in newtab:
        data2016 = data2016[data2016['HomeTeam'] != i]

for i in data2016['AwayTeam']:
    if i not in newtab:
        data2016 = data2016[data2016['AwayTeam'] != i]
        
#concat des 3 dataframe afin d'analyser tout d'un coup
frames = [data2014, data2015, data2016]
result = pd.concat(frames)
bb = result[['HomeTeam','AwayTeam','FTR']]
nbVictoires(bb, '2014+2015+2016')
nbVictoiresDomicile(bb)
nbVictoiresExterieur(bb)


Nombres de victoire par equipe pour la saison 2014+2015+2016 :
[11, 'Bochum']
[15, 'Frankfurt FSV']
[16, 'Fortuna Dusseldorf']
[19, 'Sandhausen']
[19, 'Union Berlin']
[20, 'Munich 1860']
[20, 'St Pauli']
[21, 'Greuther Furth']
[23, 'Kaiserslautern']
[25, 'Karlsruhe']
Nombres de victoire à domicile par equipe pour les 3 saisons :
[4, 'Bochum']
[7, 'Frankfurt FSV']
[10, 'Fortuna Dusseldorf']
[10, 'Munich 1860']
[11, 'Sandhausen']
[11, 'St Pauli']
[12, 'Greuther Furth']
[12, 'Karlsruhe']
[14, 'Union Berlin']
[15, 'Kaiserslautern']
Nombres de victoire à extérieur par equipe pour les 3 saisons :
[5, 'Union Berlin']
[6, 'Fortuna Dusseldorf']
[7, 'Bochum']
[8, 'Frankfurt FSV']
[8, 'Kaiserslautern']
[8, 'Sandhausen']
[9, 'Greuther Furth']
[9, 'St Pauli']
[10, 'Munich 1860']
[13, 'Karlsruhe']


In [791]:
#B365H = Bet365 home win odds
#B365D = Bet365 draw odds
#B365A = Bet365 away win odds



def rencontre(equipe1, equipe2):
    for i in bb['HomeTeam']:
        if i == equipe1:
            cv = bb[bb['HomeTeam'] == i]

    for i in cv['AwayTeam']:
        if i == equipe2:
            cv = cv[cv['AwayTeam'] == i]

    for i in bb['HomeTeam']:
        if i == equipe2:
            kj = bb[bb['HomeTeam'] == i]

    for i in kj['AwayTeam']:
        if i == equipe1:
            kj = kj[kj['AwayTeam'] == i]

    az = [cv, kj]
    za = pd.concat(az)
    return za


equipe1 = 'Frankfurt FSV'
equipe2 = 'Union Berlin'
ml = rencontre(equipe1,equipe2)
print(ml)

vicDom = 80
vicExt = 100
nulDom = 40
nulExt = 50



scoreE1 = 0
scoreE2 = 0

for index, row in ml.iterrows():
    if (row['HomeTeam'] == equipe1) & (row['FTR'] == 'H'):
        scoreE1 = scoreE1 + vicDom
    if (row['HomeTeam'] == equipe2) & (row['FTR'] == 'H'):
        scoreE2 = scoreE2 + vicDom
    if (row['HomeTeam'] == equipe1) & (row['FTR'] == 'D'):
        scoreE1 = scoreE1 + nulDom
    if (row['HomeTeam'] == equipe2) & (row['FTR'] == 'D'):
        scoreE2 = scoreE2 + nulDom
    if (row['AwayTeam'] == equipe1) & (row['FTR'] == 'A'):
        scoreE1 = scoreE1 + vicExt
    if (row['AwayTeam'] == equipe2) & (row['FTR'] == 'A'):
        scoreE2 = scoreE2 + vicExt
    if (row['AwayTeam'] == equipe1) & (row['FTR'] == 'D'):
        scoreE1 = scoreE1 + nulExt
    if (row['AwayTeam'] == equipe2) & (row['FTR'] == 'D'):
        scoreE2 = scoreE2 + nulExt



print(scoreE1)
print(scoreE2)

print(float(scoreE1)/float(scoreE1+scoreE2)*100)
 


          HomeTeam       AwayTeam FTR
42   Frankfurt FSV   Union Berlin   D
291  Frankfurt FSV   Union Berlin   A
68   Frankfurt FSV   Union Berlin   H
191   Union Berlin  Frankfurt FSV   H
142   Union Berlin  Frankfurt FSV   H
220   Union Berlin  Frankfurt FSV   H
120
390
23.5294117647


In [823]:

vbn = dataall[['HomeTeam','AwayTeam','FTR','HTR']]

def rencontre(equipe1, equipe2):
    for i in vbn['HomeTeam']:
        if i == equipe1:
            iop = vbn[vbn['HomeTeam'] == i]

    for i in iop['AwayTeam']:
        if i == equipe2:
            iop = iop[iop['AwayTeam'] == i]

    for i in vbn['HomeTeam']:
        if i == equipe2:
            tyu = vbn[vbn['HomeTeam'] == i]

    for i in tyu['AwayTeam']:
        if i == equipe1:
            tyu = tyu[tyu['AwayTeam'] == i]

    aza = [iop, tyu]
    zaz = pd.concat(aza)
    return zaz


#'Karlsruhe', 'St Pauli', 'Fortuna Dusseldorf', 'Frankfurt FSV', 'Munich 1860', 'Greuther Furth', 'Paderborn', 'Union Berlin', 'Bochum'
equipe1 = 'Karlsruhe'
equipe2 = 'Bochum'
mll = rencontre(equipe1,equipe2)
#print(mll)


vicDom = 80
vicExt = 100
vicDomMT = 20
vicExtMT = 30
nulDom = 40
nulExt = 50


scoreE1 = 0
scoreE2 = 0
 
for index, row in mll.iterrows():
    #Victoire domicile
    if (row['HomeTeam'] == equipe1) & (row['FTR'] == 'H'):
        scoreE1 = scoreE1 + vicDom
    if (row['HomeTeam'] == equipe2) & (row['FTR'] == 'H'):
        scoreE2 = scoreE2 + vicDom
        
    #Victoire MT domicile
    if (row['HomeTeam'] == equipe1) & (row['HTR'] == 'H'):
        scoreE1 = scoreE1 + vicDomMT
    if (row['HomeTeam'] == equipe2) & (row['HTR'] == 'H'):
        scoreE2 = scoreE2 + vicDomMT
        
    #Nul domicile
    if (row['HomeTeam'] == equipe1) & (row['FTR'] == 'D'):
        scoreE1 = scoreE1 + nulDom
    if (row['HomeTeam'] == equipe2) & (row['FTR'] == 'D'):
        scoreE2 = scoreE2 + nulDom
        
    #Victoire exterieur
    if (row['AwayTeam'] == equipe1) & (row['FTR'] == 'A'):
        scoreE1 = scoreE1 + vicExt
    if (row['AwayTeam'] == equipe2) & (row['FTR'] == 'A'):
        scoreE2 = scoreE2 + vicExt
    
    #Victoire exterieur MT
    if (row['AwayTeam'] == equipe1) & (row['HTR'] == 'A'):
        scoreE1 = scoreE1 + vicExtMT
    if (row['AwayTeam'] == equipe2) & (row['HTR'] == 'A'):
        scoreE2 = scoreE2 + vicExtMT
        
    #Nul exterieur
    if (row['AwayTeam'] == equipe1) & (row['FTR'] == 'D'):
        scoreE1 = scoreE1 + nulExt
    if (row['AwayTeam'] == equipe2) & (row['FTR'] == 'D'):
        scoreE2 = scoreE2 + nulExt
        
#print(scoreE1)
#print(scoreE2)

print('Le pourcentage de chance que '+ equipe1 + ' gagne dans le match ' + equipe1 + ' vs ' + equipe2 + ' est de ' + str(float(scoreE1)/float(scoreE1+scoreE2)*100) + '%')
#print('Score equipe 2 :' + str(float(scoreE2)/float(scoreE1+scoreE2)*100))

#resultat reel
hjk = data2017[['HomeTeam','AwayTeam','FTR','HTR']]
finish = ''
for index, row in hjk.iterrows():
    if (row['HomeTeam'] == equipe1) & (row['AwayTeam'] == equipe2):
        if row['FTR']=='H':
            finish = equipe1
        elif row['FTR']=='D':
            finish = 'Match nul'
        elif row['FTR']=='A':
            finish = equipe2    
print('Resultat reel : ' + finish)

Le pourcentage de chance que Karlsruhe gagne dans le match Karlsruhe vs Bochum est de 46.3917525773%
Resultat reel : Match nul


In [701]:
#Find the row numbers that should be used for training and testing.
train_idx = np.array(result.Date < '2016-01-01')
test_idx = np.array(result.Date >= '2016-01-01')
 
#Arrays where the match results are stored in
results_train = np.array(result.FTR[train_idx])
results_test = np.array(result.FTR[test_idx])

feature_columns = ['B365H', 'B365D', 'B365A', 'BWH', 'BWD', 'BWA', 'IWH',
                    'IWD', 'IWA','LBH', 'LBD', 'LBA', 'PSH', 'PSD', 'PSA',
                    'SOH', 'SOD', 'SOA', 'SBH', 'SBD', 'SBA', 'SJH', 'SJD',
                    'SJA', 'SYH', 'SYD','SYA', 'VCH', 'VCD', 'VCA', 'WHH',
                    'WHD', 'WHA']

from sklearn.preprocessing import Imputer
 
#Column numbers for odds for the three outcomes 
cidx_home = [i for i, col in enumerate(result.columns) if col[-1] in 'H' and col in feature_columns]
cidx_draw = [i for i, col in enumerate(result.columns) if col[-1] in 'D' and col in feature_columns]
cidx_away = [i for i, col in enumerate(result.columns) if col[-1] in 'A' and col in feature_columns]
 
#The three feature matrices for training
feature_train_home = result.ix[train_idx, cidx_home].as_matrix()
feature_train_draw = result.ix[train_idx, cidx_draw].as_matrix()
feature_train_away = result.ix[train_idx, cidx_away].as_matrix()
 
#The three feature matrices for testing
feature_test_home = result.ix[test_idx, cidx_home].as_matrix()
feature_test_draw = result.ix[test_idx, cidx_draw].as_matrix()
feature_test_away = result.ix[test_idx, cidx_away].as_matrix()
 
train_arrays = [feature_train_home, feature_train_draw,
                feature_train_away]
                                     
test_arrays = [feature_test_home, feature_test_draw,
                feature_test_away]
 
imputed_training_matrices = []
imputed_test_matrices = []
 
for idx, farray in enumerate(train_arrays):
    imp = Imputer(strategy='mean', axis=1) #0: column, 1:rows
    farray = imp.fit_transform(farray)
    test_arrays[idx] = imp.fit_transform(test_arrays[idx])
     
    imputed_training_matrices.append(farray)
    imputed_test_matrices.append(test_arrays[idx])
 
#merge the imputed arrays
feature_train = np.concatenate(imputed_training_matrices, axis=1)
feature_test = np.concatenate(imputed_test_matrices, axis=1)



from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
 
adb = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=3),
    n_estimators=1000,
    learning_rate=0.4, random_state=42)
 
adb = adb.fit(feature_train, results_train)

import sklearn.metrics as skm
 
training_pred = adb.predict(feature_train)
#print skm.confusion_matrix(list(training_pred), list(results_train))

test_pred = adb.predict(feature_test)
print skm.confusion_matrix(list(test_pred), list(results_test)) 


[[10 17 11]
 [ 4  1  6]
 [ 9 16 19]]



## 2.3. Analyse des cotes


## 2.4. Mise en relation des 2 analyses

# 3.Apprentissage automatisée

## 3.1. Prédiction vainqueur sur des matchs choisis (trouver des équipes qui y sont tout le temps) avec cotes ect : score comme disait guyguy ?

# 4.Conclusion